In [6]:
import os
import numpy as np
import torch
import pickle
import time
import sys

current_directory = os.getcwd()
models_dir = os.path.join(current_directory, '..')
print(models_dir)
sys.path.append(models_dir)

import torch
from torch.utils.data import DataLoader, Dataset
from models import Pose2AudioTransformer
from transformers import EncodecModel
from utils import DanceToMusic
from datetime import datetime
from torch.optim import Adam

/Users/azeez/Documents/pose_estimation/DanceToMusic/notebooks/..


/Users/azeez/miniforge3/envs/dance2music/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# assign GPU or CPU
if torch.backends.mps.is_available():
    device = torch.device("mps")
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
# device = torch.device("cpu")

model_id = "facebook/encodec_24khz"
encodec_model = EncodecModel.from_pretrained(model_id)
encodec_model.to(device)
codebook_size = encodec_model.quantizer.codebook_size
sample_rate = 24000

data_dir = "/Users/azeez/Documents/pose_estimation/DanceToMusic/data/samples/5sec_expando_dataset"
dataset = DanceToMusic(data_dir, encoder = encodec_model, sample_rate = sample_rate, device=device)
print("Dataset size: ", len(dataset))

/Users/azeez/miniforge3/envs/dance2music/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Dataset size:  1244


In [15]:
src_pad_idx = 0
trg_pad_idx = 0
learned_weights = '/Users/azeez/Documents/pose_estimation/DanceToMusic/weights/5_sec_transformer_expando__best_model_5.3600.pt'
# device = torch.device("mps")
embed_size = dataset.data['poses'].shape[2] * dataset.data['poses'].shape[3]
pose_model = Pose2AudioTransformer(codebook_size, src_pad_idx, trg_pad_idx, device=device, num_layers=4, heads = 4, embed_size=embed_size, dropout = 0.1)
pose_model.load_state_dict(torch.load(learned_weights, map_location=device))
pose_model.to(device)

Pose2AudioTransformer(
  (encoder): Encoder(
    (position_embedding): Embedding(2000, 96)
    (layers): ModuleList(
      (0-3): 4 x TransformerBlock(
        (attention): SelfAttention(
          (values): Linear(in_features=24, out_features=24, bias=False)
          (keys): Linear(in_features=24, out_features=24, bias=False)
          (queries): Linear(in_features=24, out_features=24, bias=False)
          (fc_out): Linear(in_features=96, out_features=96, bias=True)
        )
        (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (feed_forward): Sequential(
          (0): Linear(in_features=96, out_features=384, bias=True)
          (1): ReLU()
          (2): Linear(in_features=384, out_features=96, bias=True)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder): Decoder(
    (codebook_embedding): Embedding(1024

In [16]:
from IPython.display import Audio, display


def audioCodeToWav(audio_code, encodec_model, sample_rate = 24000, device='cpu'):
    audio_code = audio_code.reshape(1,1,2,int(audio_code.size(2)/2))
    audio_code = audio_code.to(device)
    audio_scale = [None]
    wav = encodec_model.decode(audio_code, audio_scale)
    return wav

In [17]:
audio_codes, pose, pose_mask, wav, wav_mask, wav_path, sr = dataset[0]
output = pose_model.generate(pose.unsqueeze(0).to(device), pose_mask.to(device), max_length = audio_codes.shape[1]+1, temperature = 1)
print(output[0][:20])
print(output.shape)
wav = audioCodeToWav(output.unsqueeze(0), encodec_model, sample_rate = 24000, device=device)['audio_values']
display(Audio(wav[0].detach().numpy(), rate=24000))

tensor([582, 793, 690, 465, 901, 415,  85, 383, 271, 758,  92, 954, 342, 602,
        685, 492,  85, 255, 722, 845])
torch.Size([1, 754])


In [22]:
from IPython.display import Video

vid_path = wav_path.replace('wav', 'mp4')
Video(vid_path)

In [41]:
from moviepy.editor import VideoFileClip, AudioFileClip
import numpy as np
from scipy.io.wavfile import write
import soundfile as sf

# Assuming `wav` is a numpy array with your new audio data
# and `vid_path` is the path to your original video file.

# Specify the path to save the output video
output_video = '../README_assets/sample_output_video.mp4'

# Create a temporary WAV file for the new audio
temp_audio_path = '/../README_assets/temp_audio.wav'
wav_np = wav[0].detach().numpy()
sf.write(file=temp_audio_path, data=wav_np, samplerate=24000, subtype='PCM_16')
print((32767/2))
scaled_wav = np.int16((wav_np * (32767/2)))
scaled_wav = scaled_wav + (scaled_wav.min() * -1)
print(scaled_wav.shape)
print(scaled_wav.max(),scaled_wav.min())
print(scaled_wav[:20])
write(temp_audio_path, 24000, scaled_wav)

video_clip = VideoFileClip(vid_path)
new_audio_clip = AudioFileClip(temp_audio_path)
video_clip_with_new_audio = video_clip.set_audio(new_audio_clip)

video_clip_with_new_audio.write_videofile(output_video, codec='libx264', audio_codec='aac')

# Close the clips to release their resources
video_clip.close()
new_audio_clip.close()
video_clip_with_new_audio.close()

# Optionally, delete the temporary audio file
os.remove(temp_audio_path)

Video(output_video)

LibsndfileError: Error opening '../README_assets/temp_audio.wav': Format not recognised.